In [87]:
import numpy as np
import time
import elpigraph
import matplotlib.pyplot as plt
import rpy2.robjects.packages as rpackages
import rpy2.robjects
import rpy2.robjects.numpy2ri
import rpy2.robjects.pandas2ri
r_elpigraph = rpackages.importr("ElPiGraph.R")
rpy2.robjects.numpy2ri.activate()
rpy2.robjects.pandas2ri.activate()
plt.style.use('seaborn')

In [2]:
run_points_colab_hybrid = [[2.235868215560913,
  8.04754900932312,
  14.218730926513672,
  22.43982243537903,
  35.0305061340332,
  51.293405294418335],
 [3.4747135639190674,
  12.28583836555481,
  19.56864619255066,
  31.423277139663696,
  46.67190456390381,
  65.93725347518921],
 [15.423911333084106,
  52.59123969078064,
  89.74802350997925,
  140.48946690559387,
  222.83035445213318,
  331.40812373161316]]

run_points = [[0.4735743999481201,
  1.8850438594818115,
  3.7966206073760986,
  6.267552375793457,
  10.309508800506592,
  16.83008885383606],
 [2.36011004447937,
  9.790740728378296,
  23.06823992729187,
  52.444769859313965,
  74.06683897972107,
  130.87490344047546],
 [30.06736946105957,
  121.04888701438904,
  258.76202154159546,
  428.54820466041565,
  910.2173552513123,
  1590.924254655838]]

run_points_r = [[3.1850550174713135,
  3.5126571655273438,
  4.8599419593811035,
  7.931446313858032,
  13.496990442276001,
  23.014589548110962],
 [4.812899112701416,
  16.89812469482422,
  30.01169180870056,
  61.17126178741455,
  103.9380316734314,
  161.51624822616577],
 [57.94627809524536,
  212.81520628929138,
  409.3852393627167,
  701.4382843971252,
  1234.1388187408447,
  2073.3358879089355]]

run_points_colab_gpu = [[11.22635293006897,
  58.06335997581482,
  165.8828558921814,
  332.5646514892578,
  576.5051832199097,
  992.215359210968],
 [63.82702708244324,
  179.68374109268188,
  337.24268078804016,
  552.5429406166077,
  893.2425599098206,
  1434.2261390686035]]

run_points_colab_gpu.append([1956.016000509262, 4283.875202894211, 5244.078868865967])

# Speed comparison

In [4]:
### Python
np.random.seed(0)
num_points = [1000,10000,100000]
num_nodes = [10,20,30,40,50,60]

run_points = []
for j in num_points:
    run_nodes = []
    for i in num_nodes:
        X=np.random.random(size=(j,10))
        s = time.time()
        res = elpigraph.computeElasticPrincipalTree(X = X,NumNodes = i,drawPCAView=False)
        end = time.time() - s
        run_nodes.append(end)
    run_points.append(run_nodes)

Generating the initial configuration
Creating a chain in the 1st PC with 2 nodes
90% of the points have been used as initial conditions. Resetting.
Constructing tree 1 of 1 / Subset 1 of 1
Performing PCA
Using standard PCA
10 dimensions are being used
100.0 % of the original variance has been retained
The elastic matrix is being used. Edge configuration will be ignored
Computing EPG with  10  nodes on  1000  points and  10  dimensions
Nodes =  2 3 4 5 6 7 8 9 

BARCODE	ENERGY	NNODES	NEDGES	NRIBS	NSTARS	NRAYS	NRAYS2	MSE	MSEP	FVE	FVEP	UE	UR	URN	URN2	URSD

1|0|0|0|0|0|0||10	0.6063	10	9	0	0	0	0	0.5868	0.5821	0.3003	0.306	0.0194	0.0	0.0004	0.0036	0


MSDEnergyPlot not yet implemented
accuracyComplexityPlot not yet implemented
0.4939  seconds elapsed
Generating the initial configuration
Creating a chain in the 1st PC with 2 nodes
90% of the points have been used as initial conditions. Resetting.
Constructing tree 1 of 1 / Subset 1 of 1
Performing PCA
Using standard PCA
10 dimensions are bein

In [5]:
### R
np.random.seed(0)
num_points = [1000,10000,100000]
num_nodes = [10,20,30,40,50,60]

run_points_r = []
for j in num_points:
    run_nodes = []
    for i in num_nodes:
        X=np.random.random(size=(j,10))
        s = time.time()
        res= r_elpigraph.computeElasticPrincipalTree(X = X,NumNodes = i)
        end = time.time() - s
        run_nodes.append(end)
    run_points_r.append(run_nodes)

[1] "Generating the initial configuration"
[1] "Creating a chain in the 1st PC with 2 nodes"
[1] "Constructing tree 1 of 1 / Subset 1 of 1"
[1] "Performing PCA on the data"
[1] "Using standard PCA"
[1] "10 dimensions are being used"
[1] "100% of the original variance has been retained"
[1] "The elastic matrix is being used. Edge configuration will be ignored"
[1] "Computing EPG with 10 nodes on 1000 points and 10 dimensions"
[1] "Using a single core"
Nodes = 2 3 4 5 6 7 8 9 
BARCODE	ENERGY	NNODES	NEDGES	NRIBS	NSTARS	NRAYS	NRAYS2	MSE	MSEP	FVE	FVEP	UE	UR	URN	URN2	URSD
1|0|0|0|0|0|0||10	0.6063	10	9	0	0	0	0	0.5868	0.5821	0.3003	0.306	0.01943	3.598e-05	0.0003598	0.003598	0
1.678 sec elapsed
[[1]]

[1] "Generating the initial configuration"
[1] "Creating a chain in the 1st PC with 2 nodes"
[1] "Constructing tree 1 of 1 / Subset 1 of 1"
[1] "Performing PCA on the data"
[1] "Using standard PCA"
[1] "10 dimensions are being used"
[1] "100% of the original variance has been retained"
[1] "The el

In [6]:
### Plotting
for i in range(len(num_points)):
    if i<2:
        plt.plot(num_nodes,np.array(run_points_colab_gpu[i])/60,marker='.')
    else:
        plt.plot(num_nodes[:3],np.array(run_points_colab_gpu[i])/60,marker='.')
    plt.plot(num_nodes,np.array(run_points_colab_hybrid[i])/60,marker='.')
    plt.plot(num_nodes,np.array(run_points[i])/60,marker='.')
    plt.plot(num_nodes,np.array(run_points_r[i])/60,marker='.')

    plt.xlabel('Number of nodes',fontsize=16)
    plt.ylabel('Time (minutes)',fontsize=16)
    plt.legend(['Python_full_gpu','Python_Hybrid_cpu_gpu','Python_one_cpu','R_one_cpu'],fontsize=13)
    plt.title('Number of points (10 dimensions) : '+str(num_points[i]),fontsize=16)
    plt.show()

NameError: name 'num_points' is not defined

# Checking output

### Step 1 :  generate output for R and Python

In [321]:
# Load data
X =  np.genfromtxt('data/tree_data.csv', delimiter=',')

# Create desired list of inputs for R and Python
input_data = [X,X,X]
epg_n_nodes = [10,20,30]
epg_lambda = [.1,.2,.3]
epg_mu = [.02,.07,.01]
epg_trimmingradius = [float('inf'),.1,.3]
epg_finalenergy = ['Penalized','Base','Penalized']
epg_alpha = [.1,.03,.05]
epg_beta = [.3,.02,.04]

# Results storage
res_py = []
res_R = []

for i in range(len(input_data)):
    res_py.append(elpigraph.computeElasticPrincipalTree(X=input_data[i],NumNodes = epg_n_nodes[i], 
                                                Lambda=epg_lambda[i], Mu=epg_mu[i],
                                                TrimmingRadius = epg_trimmingradius[i]**2,
                                                FinalEnergy = epg_finalenergy[i],
                                                alpha = epg_alpha[i],
                                                beta = epg_beta[i],                                                    
                                                Do_PCA=False,CenterData=False,
                                                n_cores = 1,
                                                nReps=1,
                                                EmbPointProb=1.0,
                                                drawPCAView=False)[0])

    tmp = r_elpigraph.computeElasticPrincipalTree(X=input_data[i],NumNodes = epg_n_nodes[i], 
                                                Lambda=epg_lambda[i], Mu=epg_mu[i],
                                                TrimmingRadius= epg_trimmingradius[i],
                                                FinalEnergy = epg_finalenergy[i],
                                                alpha = epg_alpha[i],
                                                beta = epg_beta[i],                                                    
                                                Do_PCA=False,CenterData=False,
                                                n_cores = 1,
                                                nReps=1,
                                                ProbPoint=1.0,
                                                drawPCAView=False)[0]
    res_R.append(dict(zip(tmp.names, map(list,np.array(tmp))))) # Convert R result to dict format

Generating the initial configuration
Creating a chain in the 1st PC with 2 nodes
90% of the points have been used as initial conditions. Resetting.
Constructing tree 1 of 1 / Subset 1 of 1
The elastic matrix is being used. Edge configuration will be ignored
Computing EPG with  10  nodes on  492  points and  3  dimensions
Nodes =  2 3 4 5 6 7 8 9 

BARCODE	ENERGY	NNODES	NEDGES	NRIBS	NSTARS	NRAYS	NRAYS2	MSE	MSEP	FVE	FVEP	UE	UR	URN	URN2	URSD

0||10	0.1403	10	9	8	0	0	0	0.0758	0.0731	0.8594	0.8643	0.0637	0.0008	0.0084	0.0838	0


MSDEnergyPlot not yet implemented
accuracyComplexityPlot not yet implemented
0.2404  seconds elapsed
[1] "Generating the initial configuration"
[1] "Creating a chain in the 1st PC with 2 nodes"
[1] "Constructing tree 1 of 1 / Subset 1 of 1"
[1] "The elastic matrix is being used. Edge configuration will be ignored"
[1] "Computing EPG with 10 nodes on 492 points and 3 dimensions"
[1] "Using a single core"
Nodes = 2 3 4 5 6 7 8 9 
BARCODE	ENERGY	NNODES	NEDGES	NRIBS	NST

In [328]:
%load_ext rpy2.ipython

In [333]:
np.isinf(10)

False

### Step 2 : check output NodePositions, Edges, ReportTable, FinalReport
Prints key and iteration index if a wrong result is found

In [322]:
for i in range(len(input_data)):
    one_res_py = res_py[i]
    one_res_R = res_R[i]
    for key in one_res_py:
        if key == 'NodePositions':
            try: assert np.allclose(one_res_py[key], one_res_R[key])
            except: print(key,i)

        if key == 'Edges':
            try: assert all(map(lambda x:np.all(x),[one_res_py[key][0]==(one_res_R[key][0]-1), #correcting R indexing that starts at one
                                              one_res_py[key][1]==one_res_R[key][1],
                                              one_res_py[key][2]==one_res_R[key][2]]))
            except: print(key,i)

        if key == 'ReportTable':
            # This messy loop procedure is because I round to 4 digits in my version for nicer printing...
            # R prints more decimals
            flattened_py = [j for e in list(one_res_py[key].values()) for j in e]
            tmp = []
            for a,b in list(zip(flattened_py,one_res_R['ReportTable'])):
                if len(b)<=6:
                    tmp.append(a==b)
                else:
                    eps = 1e-10
                    tmp.append(float(a)==(np.round(float(b)+eps,4)))    # adding eps to round up for edge case .5          
            try: assert np.all(np.array(tmp))
            except: print(key,i)

        if key == 'FinalReport':
            try: assert(np.allclose(np.array(list(one_res_py['FinalReport'].values()))[1:].astype(float), 
                                    np.array(one_res_R['FinalReport']).flatten()[1:].astype(float)))
            except: print(key,i)

NodePositions 1
Edges 1


ValueError: could not convert string to float: '0||10'

In [323]:
res_py[1]['NodePositions']

array([[-0.0128871 , -0.02502565,  0.02561351],
       [-0.0158272 , -0.02717825,  0.02588762],
       [-0.01875965, -0.02937826,  0.02616736],
       [-0.02169165, -0.03158106,  0.02644743],
       [-0.02462365, -0.03378385,  0.02672751],
       [-0.02755565, -0.03598665,  0.02700758],
       [-0.03048766, -0.03818945,  0.02728766],
       [-0.03341964, -0.04039223,  0.02756773],
       [-0.03635147, -0.0425949 ,  0.02784779],
       [-0.03927704, -0.04479286,  0.02812725],
       [-0.00928441, -0.02697979,  0.02582779],
       [-0.01289565, -0.02497267,  0.02560721],
       [-0.01289565, -0.02497267,  0.02560721],
       [-0.00567391, -0.02898385,  0.02604801],
       [-0.04190007, -0.04676354,  0.02837781],
       [-0.01289564, -0.02497271,  0.02560721],
       [-0.00208272, -0.03097848,  0.02626721],
       [-0.01289541, -0.02497414,  0.02560738],
       [ 0.00116638, -0.03278312,  0.02646552],
       [-0.01289565, -0.02497267,  0.02560721]])

In [324]:
res_R[1]['NodePositions']

[array([-0.0039198 , -0.70612129,  0.02589117]),
 array([-0.00274325, -0.69702193,  0.02537736]),
 array([-0.00156554, -0.68792352,  0.02485943]),
 array([-3.01469695e-04, -6.78792972e-01,  2.44000121e-02]),
 array([-0.00148347, -0.66979025,  0.02376041]),
 array([-0.00212181, -0.66074862,  0.02329673]),
 array([-0.00133628, -0.65162219,  0.0227718 ]),
 array([-2.24666764e-04, -6.42517070e-01,  2.20241449e-02]),
 array([-0.00503587, -0.71527051,  0.02619021]),
 array([ 1.60695855e-04, -6.33334311e-01,  2.14075433e-02]),
 array([-1.06217223e-04, -6.24111509e-01,  2.12832077e-02]),
 array([-0.00614341, -0.72442177,  0.02648674]),
 array([-0.00684143, -0.73363515,  0.02685948]),
 array([ 1.62242658e-04, -6.14945564e-01,  2.12716711e-02]),
 array([-0.00682454, -0.74288466,  0.02719848]),
 array([-0.00667168, -0.7521119 ,  0.02788518]),
 array([ 2.36303551e-04, -6.05798502e-01,  2.14416284e-02]),
 array([-0.00733014, -0.76131664,  0.0285622 ]),
 array([ 0.00078749, -0.59668158,  0.02142035]